### 卷积神经网络

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as fn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline 
# %matplotlib inline 的作用是 将 matplotlib 生成的图表直接嵌入到笔记本中 无需使用 plt.show() 就能显示图表 提高交互性，方便查看和分析数据可视化结果 
# 这个魔法命令使得数据分析和可视化工作更加流畅和直观。它特别适合于探索性数据分析和快速原型设计

### 读取数据
- 分别构建训练集和测试集(验证集)
- DataLoader来迭代取数据

In [26]:
input_size = 28  # 图像的总尺寸28*28
num_classes = 10  # 标签的种类数
num_epochs = 3  # 训练的总循环周期
batch_size = 64  # 一个撮(批次)的大小,64张图片

# 训练集
train_dataset = datasets.MNIST(root='../data',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

# 测试集
test_dataset = datasets.MNIST(root='../data',
                              train=False,
                              transform=transforms.ToTensor())

# 构建batch数据
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=True)

### 卷积网络模块构建
- 一般卷积层,relu层,池化层可以写成一个套餐
- 注意卷积的最后结果还是一个特征图,需要把图转换成向量才能做分类或者回归任务

In [27]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(  # 输入大小(1x28x28)
            nn.Conv2d(
                in_channels=1,  # 灰度图
                out_channels=16,  # 要得到多少个特征图
                kernel_size=5,  # 卷积核大小
                stride=1,  # 步长
                padding=2  # 如果希望卷积后跟原来大小一样,需要设置padding=(kernel_size-1)/2 if stride =1
            ),  # 输出特征图为(16,28,28)
            nn.ReLU(),  # relu层
            nn.MaxPool2d(kernel_size=2)  # 输出 (32,7,7)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.out = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)  # flatten操作，结果为：(batch_size, 32 * 7 * 7)
        return self.out(x)

### 准确率评估标准

In [28]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)

### 训练网络模型

In [30]:
# 实例化
net = CNN()
# 损失函数
criterion = nn.CrossEntropyLoss()
# 优化器 普通的随机梯度下降算法
optimizer = optim.Adam(net.parameters(), lr=0.001)
#开始训练循环
for epoch in range(num_epochs):
    #当前epoch的结果保存下来
    train_rights = [] 
    
    for batch_idx, (data, target) in enumerate(train_loader):  #针对容器中的每一个批进行循环
        net.train()                             
        output = net(data) 
        loss = criterion(output, target) 
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 
        right = accuracy(output, target) 
        train_rights.append(right) 

    
        if batch_idx % 100 == 0: 
            
            net.eval() 
            val_rights = [] 
            
            for (data, target) in test_loader:
                output = net(data) 
                right = accuracy(output, target) 
                val_rights.append(right)
                
            #准确率计算
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))

            print('当前epoch: {} [{}/{} ({:.0f}%)]\t损失: {:.6f}\t训练集准确率: {:.2f}%\t测试集正确率: {:.2f}%'.format(
                epoch, batch_idx * batch_size, len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss.data, 
                100. * train_r[0].numpy() / train_r[1], 
                100. * val_r[0].numpy() / val_r[1]))


当前epoch: 0 [0/60000 (0%)]	损失: 2.300769	训练集准确率: 12.50%	测试集正确率: 11.79%
当前epoch: 0 [6400/60000 (11%)]	损失: 0.404928	训练集准确率: 77.46%	测试集正确率: 92.15%
当前epoch: 0 [12800/60000 (21%)]	损失: 0.230376	训练集准确率: 85.16%	测试集正确率: 94.64%
当前epoch: 0 [19200/60000 (32%)]	损失: 0.352231	训练集准确率: 88.46%	测试集正确率: 96.43%
当前epoch: 0 [25600/60000 (43%)]	损失: 0.040155	训练集准确率: 90.39%	测试集正确率: 97.27%
当前epoch: 0 [32000/60000 (53%)]	损失: 0.102461	训练集准确率: 91.64%	测试集正确率: 97.23%
当前epoch: 0 [38400/60000 (64%)]	损失: 0.048507	训练集准确率: 92.55%	测试集正确率: 97.65%
当前epoch: 0 [44800/60000 (75%)]	损失: 0.063543	训练集准确率: 93.25%	测试集正确率: 97.88%
当前epoch: 0 [51200/60000 (85%)]	损失: 0.168339	训练集准确率: 93.81%	测试集正确率: 97.56%
当前epoch: 0 [57600/60000 (96%)]	损失: 0.098812	训练集准确率: 94.26%	测试集正确率: 98.04%
当前epoch: 1 [0/60000 (0%)]	损失: 0.050791	训练集准确率: 96.88%	测试集正确率: 98.37%
当前epoch: 1 [6400/60000 (11%)]	损失: 0.043483	训练集准确率: 98.19%	测试集正确率: 98.26%
当前epoch: 1 [12800/60000 (21%)]	损失: 0.048166	训练集准确率: 98.20%	测试集正确率: 98.31%
当前epoch: 1 [19200/60000 (32%)]	损失: 0.057461	训练集准确率